In [ ]:
using Polyhedra
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector
import MechanismComplementarity
using JuMP, Gurobi

In [ ]:
reload("MechanismComplementarity")
mc = MechanismComplementarity

In [ ]:
mechanism = parse_urdf(Float64, "box_valkyrie.urdf")
world = root_body(mechanism)
floating_base = joint_to_parent(findbody(mechanism, "core"), mechanism)
floating_base.jointType = QuaternionFloating{Float64}()

vis = Visualizer()[:boxval]
setgeometry!(vis, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))
inspect(mechanism, vis)

In [ ]:
floor = mc.Obstacle(
    default_frame(world),
    SimpleHRepresentation{3, Float64}([0 0 1], [0]),
    HalfSpace{3, Float64}([0, 0, 1], 0),
    0.5)

free_space = mc.FreeRegion(
    default_frame(world),
    SimpleHRepresentation{3, Float64}(
        [0 0 -1],
        [0])
    )

contact_limbs = findbody.(mechanism, ["rh", "lh", "rf", "lf"])

env = mc.Environment(
    Dict([body => mc.ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [floor],
                [free_space])
            for body in contact_limbs]));

In [ ]:
limits = Dict([
    findjoint(mechanism, "core_to_rh_x") => SimpleHRepresentation([1. -1]', [1.0, -0.5]),
    findjoint(mechanism, "core_to_lh_x") => SimpleHRepresentation([1. -1]', [1.0, -0.5]),
    findjoint(mechanism, "core_to_rh_z") => SimpleHRepresentation([1. -1]', [0.5, 0.5]),
    findjoint(mechanism, "core_to_lh_z") => SimpleHRepresentation([1. -1]', [0.5, 0.5]),
    findjoint(mechanism, "core_to_rf_x") => SimpleHRepresentation([1. -1]', [0.5, 0.0]),
    findjoint(mechanism, "core_to_lf_x") => SimpleHRepresentation([1. -1]', [0.5, 0.0]),
    findjoint(mechanism, "core_to_rf_z") => SimpleHRepresentation([1. -1]', [-0.5, 1.0]),
    findjoint(mechanism, "core_to_lf_z") => SimpleHRepresentation([1. -1]', [-0.5, 1.0]),
        ])

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_velocity!(x0, floating_base, [0, 0, 0, 0, 0, 0])
set_configuration!(x0, floating_base, [1, 0, 0, 0, 0, 0, 1])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-0.9])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-0.9])

In [ ]:
settransform!(vis, x0)

In [ ]:

controller = x -> begin
    kp = 20
    kd = 0.1 * kp
    u = kp .* (configuration(x0) .- configuration(x))[5:end] .- kd .* velocity(x)[4:end]
    vcat(zeros(3), u)
end

Δt = 0.05
results = mc.simulate(x0, controller, limits, env, Δt, 120, GurobiSolver(OutputFlag=0));

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, floating_base, [1, 0, 0, 0, 0, 0, 1.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-1])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-1])

In [ ]:
Δt = 0.05
input_limits = Dict([
        joint=>[-10, 10]
    for joint in keys(limits)])

model, results_opt = mc.optimize(x0, input_limits, limits, env, Δt, 7, Model(solver=GurobiSolver(TimeLimit=60)))
@objective model Min -100 * configuration(results_opt[end].state, floating_base)[end] + sum([0.001 * sum(r.input.^2) for r in results_opt])
for i in 1:length(results_opt)
    @constraint(model, results_opt[i].state.q[1:4] .== [1, 0, 0, 0])
end
solve(model)
results_opt = getvalue.(results_opt);

In [ ]:
for r in results_opt
    settransform!(vis, r.state)
    sleep(0.2)
end